<a href="https://colab.research.google.com/github/kchen79427/Capstone_Bruestle_Boudia_Chen/blob/dave_branch/Modeling/Weather_Boudia_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=8a04af6feef6a5f60c35c4b348d05f2ec6e5f988dbef43e86d3e34364f4fb726
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [195]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, Reshape, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
from pyspark.sql.functions import to_timestamp
from pyspark.sql.types import TimestampType
import json
from datetime import datetime

In [7]:
spark =  SparkSession.builder.master("local[*]") \
  .appName('Takeoff') \
  .getOrCreate()

# Helper Functions

In [52]:
# load json object for referencing filepaths
files = json.load(open('drive/Shareddrives/STUDENT-Capstone SS23/files.json','r'))

In [218]:
def createFlightDT(fdate, ftime):
  # Creates a date-time timestamp
  lst = [x for x in ftime]
  lst.insert(2,':') # insert a column to delineate hour from min
  fdt = fdate + ' ' + "".join(lst) + ':00'
  fdt = datetime.strptime(fdt, '%Y-%m-%d %H:%M:%S')
  return fdt

# Load Weather Training Data

In [88]:
# Load Weather data from parquet into spark DataFrame
weather = spark.read.parquet(files['Weather']['Origin'])
weather.createOrReplaceTempView('All_Weather')

In [90]:
# If getting DFW Weather Specifically
DFW_weather = spark.sql("Select * FROM All_Weather WHERE Origin = 'DFW'")
DFW_weather.createOrReplaceTempView('weather')

# Load Flight Data

In [219]:
spdf = spark.read.parquet(files['BTS']['OnTime'])

In [220]:
# Create a unique Flight ID for each flight for easy reference
dfw = spdf.withColumn('Flight_Num', col('Reporting_Airline') + \
                    col('Flight_Number_Reporting_Airline') + \
                    col('FlightDate'))

In [221]:
# Create a Flight Date/Time Timestamp to be able to compare to the Weather Dataset
func = udf(lambda x,y: createFlightDT(x,y))
dfw = dfw.withColumn('Flight_DateTime', to_timestamp(func(dfw.FlightDate, dfw.DepTime)))

In [222]:
dfw.createOrReplaceTempView('Ontime')

In [234]:
dfw.columns

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Reporting_Airline',
 'DOT_ID_Reporting_Airline',
 'IATA_CODE_Reporting_Airline',
 'Tail_Number',
 'Flight_Number_Reporting_Airline',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'Origin',
 'OriginCityName',
 'OriginState',
 'OriginStateFips',
 'OriginStateName',
 'OriginWac',
 'DestAirportID',
 'DestAirportSeqID',
 'DestCityMarketID',
 'Dest',
 'DestCityName',
 'DestState',
 'DestStateFips',
 'DestStateName',
 'DestWac',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRSArrTime',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'ArrDel15',
 'ArrivalDelayGroups',
 'ArrTimeBlk',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'AirTime',
 'Flights',
 'Distance',
 'DistanceGroup',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityD

In [241]:
temp = spark.sql("""
                  Select
                    Reporting_Airline,
                    Flight_Number_Reporting_Airline,
                    FlightDate,
                    DepDelayMinutes,
                    DepDelay,
                    DepTime,
                    ArrTime
                  FROM
                    OnTime
                  WHERE
                    Origin = 'DFW' AND
                    DepTime = ''

          """)



In [244]:
temp.count()

18137

# Combine Weather Data with Flight

In [57]:
result = spark.sql("SELECT ")

<function __main__.createFlightDT(flightdate, deptime)>

In [64]:
upd = spark.sql('''SELECT to_timestamp(create_flight_dt_sql(FlightDate,DepTime)) FROM OnTime LIMIT 5
          ''')

In [81]:
spark.sql('Select Flight_DT FROM OnTime')

DataFrame[Flight_DT: timestamp]

In [69]:
spark_weather.createOrReplaceTempView('Weather')

In [70]:
rslts = spark.sql('Select * FROM Weather Limit 1')

In [71]:
rslts.show()

+------+----------+----------+-------------------+-------------------+-----------+----------+------+------------+------------+---------+----------+---------+-----------+--------+-------------+-------+------------+-------+------------+-------+------------+---------+--------------+--------------+--------------+---------------+---------------+-------+------------+----------------+-----------------+-----------+----------------+-----------+-----------+----------------+---------------+--------------------+--------------------+-------------------------+----------------------+----------------+--------------------+
|Origin|FlightDate|CRSDepTime|         OriginDtTm|        WeatherDtTm|w_dir_angle|w_dir_qlty|w_type|w_speed_rate|w_speed_qlty|sky_c_hgt|sky_c_qlty|sky_c_det|sky_c_cavok|vis_dist|vis_dist_qlty|vis_var|vis_var_qlty|tmp_air|tmp_air_qlty|tmp_dew|tmp_dew_qlty|sea_lvl_p|sea_lvl_p_qlty|liq_precip_qty|liq_precip_dim|liq_precip_cond|liq_precip_qlty|sky_cov|sky_cov_qlty|sky_cov_base_hgt|sky_co